In [7]:
from json import load, dump

# load and process the Midas taxonomy
DADA_taxonomy = load(open("../../sludge/DADA2_taxonomy.fa MiDAS 5.3.json", "r"))
print(len(DADA_taxonomy))
midas_taxonomy = {}
for k, v in DADA_taxonomy.items():
    midas_taxonomy[v] = {}
    taxa = k.split(";")
    midas_taxonomy[v]["kingdom"] = taxa[0]
    midas_taxonomy[v]["phylum"] = taxa[1]
    midas_taxonomy[v]["class"] = taxa[2]
    midas_taxonomy[v]["order"] = taxa[3]
    midas_taxonomy[v]["family"] = taxa[4]
    midas_taxonomy[v]["genus"] = taxa[5]
    midas_taxonomy[v]["species"] = taxa[6]

print(len(midas_taxonomy))
# export the processed Midas taxonomy
dump(midas_taxonomy, open("processed_midas_taxonomy.json", "w"), indent=3)

40195
40195


# matching the experimental 16S sequences to the Midas organisms

In [ ]:
for 

# finding RefSeq IDs for these closest organisms

In [ ]:
# get the UIDs for all sequences
search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "assembly",
    "retmax": "100000",
    "retmode": "json"
}

def get_uids(taxa):
    genus = taxa["genus"].replace("_", " ").replace("Ca", "Candidatus")
    species = taxa["species"].replace("_", " ").replace("Ca", "Candidatus")
    if "midas" in species: params.update({"term": f"{genus}[Organism]"})
    else: params.update({"term": f"{genus}+{species}[Organism]"})
    return requests.get(search_url, params=params).json()["esearchresult"]["idlist"]

# get the assemblies for all UIDs
def get_accessions(batch):
    summary_params = {
        "db": "assembly",
        "id": ",".join(batch),
        "retmode": "json"
    }
    summaries = requests.get(summary_url, params=summary_params).json()["result"]
    accessions = []
    for uid in batch:
        record = summaries[uid]
        accessions.append({
            "accession": record["assemblyaccession"],
            "organism": record["organism"],
            "taxid": record["taxid"]
        })
    return accessions

from numpy import array_split, ceil
from itertools import chain

uids = list(chain.from_iterable([get_uids(taxa) for seq, taxa in midas_taxonomy.items()]))
num_batches = ceil(len(uids)/500)
all_accessions = list(chain.from_iterable([get_accessions(batch) for batch in array_split(uids, num_batches)]))